# 🚁 Drone EXIF Extraction Demo

This notebook demonstrates how to:
- List and access drone images stored in Snowflake internal stages
- Extract EXIF metadata including GPS coordinates from drone imagery
- Store metadata in a Snowflake database table
- Visualize drone flight paths on an interactive map

**Prerequisites:** Run the `setup_drone_demo.sql` script to create the database, stage, and tables.


Packages to add to the notebook: pillow and pydeck

In [ ]:
# Import required libraries and start active session
import streamlit as st
import pandas as pd
import numpy as np
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, lit
from snowflake.snowpark.types import StructType, StructField, StringType, FloatType, IntegerType, TimestampType, VariantType
from snowflake.snowpark.files import SnowflakeFile  # Required for reading files from stage
import json
import io  # Required for reading image streams from Snowflake stages
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
from datetime import datetime
from snowflake.snowpark.context import get_active_session

session = get_active_session()

In [ ]:
USE ROLE SYSADMIN;

In [ ]:
# Initialize Snowflake session (automatically available in Snowflake Notebooks)
# session is pre-configured in Snowflake environment

st.write("### ❄️ Snowflake Session")
st.write(f"✅ Connected to Snowflake")
st.write(f"📍 Current Database: {session.get_current_database()}")
st.write(f"📍 Current Schema: {session.get_current_schema()}")
st.write(f"👤 Current Role: {session.get_current_role()}")


## 2. List Files in Stage


In [ ]:
# List all files in the drone images stage
stage_files = session.sql("LIST @DRONE_IMAGES_STAGE").collect()

print(f"📁 Found {len(stage_files)} files in DRONE_IMAGES_STAGE")
print("\nFirst 10 files:")

# Convert to pandas DataFrame for better display
if stage_files:
    stage_df = pd.DataFrame([row.asDict() for row in stage_files])
    
    # Display file information
    st.write("### Stage File Listing")
    st.dataframe(stage_df.head(10))
    
    # Show summary statistics
    total_size_mb = stage_df['size'].sum() / (1024 * 1024)
    st.metric("Total Files", len(stage_df))
    st.metric("Total Size (MB)", f"{total_size_mb:.2f}")
    
    print(f"Total files: {len(stage_df)}")
    print(f"Total size: {total_size_mb:.2f} MB")
else:
    st.warning("No files found in stage. Please upload drone images first.")
    print("⚠️ No files found in stage. Please upload drone images using SnowCLI:")
    print('snow stage copy "DroneImages/*.JPG" @DRONE_IMAGES_STAGE')


## 3. Helper Functions


In [ ]:
def download_and_display_image(file_name):
    """Download an image from stage and display it - Snowflake compatible version"""
    try:
        # Extract just the filename if it contains a path
        if '/' in file_name:
            filename_only = file_name.split('/')[-1]
        else:
            filename_only = file_name
            
        # Read image from stage using SnowflakeFile (filename only approach)
        with SnowflakeFile.open(f"@DRONE_IMAGES_STAGE/{filename_only}", 'rb') as f:
            image_data = f.read()
        
        # Create PIL Image from the stream data
        image = Image.open(io.BytesIO(image_data))
        
        # Display image info
        st.image(image, caption=f"Drone Image: {file_name}", use_column_width=True)
        
        # Show image properties
        col1, col2, col3 = st.columns(3)
        with col1:
            st.metric("Width", image.width)
        with col2:
            st.metric("Height", image.height)
        with col3:
            st.metric("Mode", image.mode)
        
        return image
        
    except Exception as e:
        st.error(f"Error loading image {file_name}: {str(e)}")
        return None

def convert_gps_to_degrees(gps_coordinate, gps_ref):
    """Convert GPS coordinates from EXIF format to decimal degrees"""
    try:
        degrees = float(gps_coordinate[0])
        minutes = float(gps_coordinate[1])
        seconds = float(gps_coordinate[2])
        
        decimal_degrees = degrees + (minutes / 60.0) + (seconds / 3600.0)
        
        if gps_ref in ['S', 'W']:
            decimal_degrees = -decimal_degrees
            
        return decimal_degrees
    except:
        return None

def extract_exif_data_from_stage_file(file_name):
    """Extract comprehensive EXIF data from image file in Snowflake stage"""
    try:
        # Extract just the filename if it contains a path
        if '/' in file_name:
            filename_only = file_name.split('/')[-1]
        else:
            filename_only = file_name
            
        # Read image from stage using SnowflakeFile (filename only approach)
        with SnowflakeFile.open(f"@DRONE_IMAGES_STAGE/{filename_only}", 'rb') as f:
            image_data = f.read()
        
        # Process image and extract EXIF
        image = Image.open(io.BytesIO(image_data))
        exif_data = image._getexif()
        
        if not exif_data:
            return None
            
        # Initialize metadata dictionary
        metadata = {
            'raw_exif': {},
            'gps_data': {},
            'camera_info': {},
            'image_info': {},
            'datetime_info': {}
        }
        
        # Process EXIF tags
        for tag_id, value in exif_data.items():
            tag = TAGS.get(tag_id, tag_id)
            metadata['raw_exif'][tag] = str(value)
            
            # Extract specific metadata categories
            if tag == 'GPSInfo':
                gps_data = {GPSTAGS.get(key, key): val for key, val in value.items()}
                metadata['gps_data'] = gps_data
                
            elif tag in ['Make', 'Model']:
                metadata['camera_info'][tag] = value
                
            elif tag in ['ExifImageWidth', 'ExifImageHeight', 'Orientation']:
                metadata['image_info'][tag] = value
                
            elif tag in ['DateTime', 'DateTimeOriginal', 'DateTimeDigitized']:
                metadata['datetime_info'][tag] = value
        
        return metadata
        
    except Exception as e:
        print(f"Error extracting EXIF from {file_name}: {str(e)}")
        return None

st.write("✅ All helper functions defined")


## 4. Sample Image Viewer


In [ ]:
# Display a sample drone image
st.write("### 📸 Sample Image Viewer")

if 'stage_df' in globals() and not stage_df.empty:
    # Find JPG files
    jpg_files = [name for name in stage_df['name'] if name.lower().endswith('.jpg')]
    
    if jpg_files:
        # Let user select an image to view
        selected_file = st.selectbox("Select an image to view:", jpg_files[:10])  # Show first 10 for demo
        
        if st.button("View Selected Image"):
            image = download_and_display_image(selected_file)
            if image:
                st.success("✅ Image loaded successfully!")
    else:
        st.warning("⚠️ No JPG files found in stage")
else:
    st.info("ℹ️ Please run the stage listing section first")


In [ ]:
def process_drone_images_batch():
    """Process a batch of drone images and extract metadata - Snowflake compatible version"""
    try:
        # Check if stage_df exists in the global scope
        if 'stage_df' not in globals() or stage_df.empty:
            st.error("Please run the stage listing cell first.")
            return None
    except NameError:
        st.error("Please run the stage listing cell first.")
        return None
        
    # Get list of image files
    image_files = [name for name in stage_df['name'] if name.lower().endswith('.jpg')][:50]  # Process first 5 for demo
    
    if not image_files:
        st.error("No JPG files found in stage.")
        return None
    
    metadata_records = []
    
    progress_bar = st.progress(0)
    status_text = st.empty()
    
    for i, file_name in enumerate(image_files):
        status_text.text(f"Processing {file_name}...")
        progress_bar.progress((i + 1) / len(image_files))
        
        try:
            # Extract EXIF data directly from stage file
            exif_data = extract_exif_data_from_stage_file(file_name)
            
            if exif_data:
                # Process GPS coordinates
                gps_lat = None
                gps_lon = None
                gps_alt = None
                
                if exif_data['gps_data']:
                    gps_info = exif_data['gps_data']
                    
                    if 'GPSLatitude' in gps_info and 'GPSLatitudeRef' in gps_info:
                        gps_lat = convert_gps_to_degrees(gps_info['GPSLatitude'], gps_info['GPSLatitudeRef'])
                        
                    if 'GPSLongitude' in gps_info and 'GPSLongitudeRef' in gps_info:
                        gps_lon = convert_gps_to_degrees(gps_info['GPSLongitude'], gps_info['GPSLongitudeRef'])
                        
                    if 'GPSAltitude' in gps_info:
                        gps_alt = float(gps_info['GPSAltitude'])
                
                # Create metadata record
                record = {
                    'FILE_NAME': file_name,
                    'FILE_PATH': f"@DRONE_IMAGES_STAGE/{file_name}",  # Full stage path
                    'FILE_SIZE': stage_df[stage_df['name'] == file_name]['size'].iloc[0] if not stage_df[stage_df['name'] == file_name].empty else None,
                    'UPLOAD_TIMESTAMP': datetime.now(),
                    'CAMERA_MAKE': exif_data['camera_info'].get('Make'),
                    'CAMERA_MODEL': exif_data['camera_info'].get('Model'),
                    'GPS_LATITUDE': gps_lat,
                    'GPS_LONGITUDE': gps_lon,
                    'GPS_ALTITUDE': gps_alt,
                    'GPS_ALTITUDE_REF': exif_data['gps_data'].get('GPSAltitudeRef') if exif_data['gps_data'] else None,
                    'IMAGE_WIDTH': exif_data['image_info'].get('ExifImageWidth'),
                    'IMAGE_HEIGHT': exif_data['image_info'].get('ExifImageHeight'),
                    'ORIENTATION': exif_data['image_info'].get('Orientation'),
                    'FOCAL_LENGTH': exif_data['raw_exif'].get('FocalLength'),
                    'APERTURE_VALUE': exif_data['raw_exif'].get('ApertureValue'),
                    'ISO_SPEED': exif_data['raw_exif'].get('ISOSpeedRatings'),
                    'EXPOSURE_TIME': exif_data['raw_exif'].get('ExposureTime'),
                    'WHITE_BALANCE': exif_data['raw_exif'].get('WhiteBalance'),
                    'DATETIME_ORIGINAL': exif_data['datetime_info'].get('DateTimeOriginal'),
                    'DATETIME_DIGITIZED': exif_data['datetime_info'].get('DateTimeDigitized'),
                    'DRONE_MODEL': exif_data['raw_exif'].get('Model'),  # Often contains drone model
                    'FLIGHT_ALTITUDE': gps_alt,  # Use GPS altitude as flight altitude
                    'GIMBAL_ROLL': exif_data['raw_exif'].get('GimbalRollDegree'),
                    'GIMBAL_YAW': exif_data['raw_exif'].get('GimbalYawDegree'), 
                    'GIMBAL_PITCH': exif_data['raw_exif'].get('GimbalPitchDegree'),
                    'RAW_EXIF_DATA': exif_data['raw_exif']  # Keep as dict, not JSON string for VARIANT column
                }
                
                metadata_records.append(record)
                
        except Exception as e:
            st.warning(f"Error processing {file_name}: {str(e)}")
    
    status_text.text("Processing complete!")
    return metadata_records

st.write("✅ Batch processing function defined")


## 5. EXIF Metadata Extraction


In [ ]:
# Run EXIF extraction
st.write("### 🔍 EXIF Metadata Extraction")

if st.button("Extract EXIF Data from Sample Images"):
    with st.spinner("Extracting EXIF metadata..."):
        metadata_records = process_drone_images_batch()
        
        if metadata_records:
            # Display results
            metadata_df = pd.DataFrame(metadata_records)
            st.write(f"📊 Extracted metadata from {len(metadata_records)} images")
            st.dataframe(metadata_df)
            
            # Store in session state for map display and database saving
            st.session_state['metadata_df'] = metadata_df
            
            # Show GPS coordinate summary
            gps_records = metadata_df.dropna(subset=['GPS_LATITUDE', 'GPS_LONGITUDE'])
            st.success(f"✅ Found GPS coordinates in {len(gps_records)} images")
        else:
            st.error("No metadata could be extracted from the images.")


## 6. Save Metadata to Snowflake Table


In [ ]:
# Save extracted metadata to database
st.write("### 💾 Save EXIF Data to Snowflake")

if st.button("Save Metadata to Database"):
    # Check if we have metadata to save
    if 'metadata_df' in st.session_state and not st.session_state.metadata_df.empty:
        try:
            # Convert pandas DataFrame to Snowpark DataFrame
            snowpark_df = session.create_dataframe(st.session_state.metadata_df)
            
            # Write to the DRONE_METADATA table
            snowpark_df.write.mode("overwrite").save_as_table("DRONE_METADATA")
            
            st.success(f"✅ Successfully saved {len(st.session_state.metadata_df)} records to DRONE_METADATA table")
            
            # Show confirmation query
            st.write("**Verify data in database:**")
            verification_df = session.sql("SELECT COUNT(*) as TOTAL_RECORDS FROM DRONE_METADATA").collect()
            st.metric("Total Records in Database", verification_df[0]['TOTAL_RECORDS'])
            
        except Exception as e:
            st.error(f"❌ Error saving to database: {str(e)}")
    else:
        st.warning("⚠️ No metadata available to save. Please extract EXIF data first.")

# Display current database contents
st.write("### 📊 Current Database Contents")
if st.button("View Database Records"):
    try:
        db_records = session.sql("""
            SELECT 
                file_name, 
                gps_latitude, 
                gps_longitude, 
                gps_altitude,
                camera_make,
                camera_model,
                datetime_original
            FROM DRONE_METADATA 
            ORDER BY datetime_original DESC
            LIMIT 20
        """).collect()
        
        if db_records:
            db_df = pd.DataFrame([row.asDict() for row in db_records])
            st.dataframe(db_df)
            st.success(f"📈 Showing latest 20 records")
        else:
            st.info("📭 No records found in database yet.")
            
    except Exception as e:
        st.error(f"❌ Error querying database: {str(e)}")


## 7. GPS Coordinate Map Display


In [ ]:
# Display drone locations - Snowflake compatible version
st.write("### 🗺️ Drone Flight Locations")

if 'metadata_df' in st.session_state and not st.session_state.metadata_df.empty:
    # Filter records with GPS coordinates
    gps_records = st.session_state.metadata_df.dropna(subset=['GPS_LATITUDE', 'GPS_LONGITUDE'])
    
    if not gps_records.empty:
        # Show summary statistics
        col1, col2, col3, col4 = st.columns(4)
        with col1:
            st.metric("📸 Images with GPS", len(gps_records))
        with col2:
            center_lat = gps_records['GPS_LATITUDE'].mean()
            st.metric("🌍 Avg Latitude", f"{center_lat:.6f}")
        with col3:
            center_lon = gps_records['GPS_LONGITUDE'].mean()
            st.metric("🌍 Avg Longitude", f"{center_lon:.6f}")
        with col4:
            st.metric("⛰️ Avg Altitude", f"{gps_records['GPS_ALTITUDE'].mean():.1f}m")
        
        # Display interactive map - Snowflake compatible
        st.write("### 🗺️ Drone Flight Locations Map")
        
        # Prepare data for st.map
        map_data = gps_records.copy()
        map_data['lat'] = map_data['GPS_LATITUDE']
        map_data['lon'] = map_data['GPS_LONGITUDE']
        
        # Add slight random offset to separate overlapping points
        import numpy as np
        np.random.seed(42)  # For consistent results
        offset = 0.00002  # Very small offset (~2 meters)
        map_data['lat'] = map_data['lat'] + np.random.uniform(-offset, offset, len(map_data))
        map_data['lon'] = map_data['lon'] + np.random.uniform(-offset, offset, len(map_data))
        
        # Display the map
        st.map(map_data[['lat', 'lon']], zoom=16)
        
        st.info("💡 **Map Tip:** Points have been slightly offset to prevent overlapping while maintaining accurate location representation.")
        
        # Display GPS coordinates table
        st.write("### 📍 GPS Coordinates Details")
        display_cols = ['FILE_NAME', 'GPS_LATITUDE', 'GPS_LONGITUDE', 'GPS_ALTITUDE', 'CAMERA_MAKE', 'CAMERA_MODEL', 'DATETIME_ORIGINAL']
        available_cols = [col for col in display_cols if col in gps_records.columns]
        st.dataframe(gps_records[available_cols], use_container_width=True)
            
    else:
        st.warning("⚠️ No GPS coordinates found in the extracted metadata.")
else:
    st.info("ℹ️ Please extract EXIF metadata first to display locations.")

st.write("🎉 **Demo Complete!** You have successfully processed drone imagery, extracted EXIF metadata, saved it to Snowflake, and displayed the GPS coordinates with clickable Google Maps links.")
